# Data Aggregation

In [106]:
import os
os.getcwd()


'C:\\Users\\User\\OneDrive - Universiti Kebangsaan Malaysia\\Sem 1\\STQD6014 Data Science\\Python Code'

In [105]:
import os
os.chdir("C:/Users/User/OneDrive - Universiti Kebangsaan Malaysia/Sem 1/STQD6014 Data Science/Python Code")


In [1]:
import numpy as np
import pandas as pd

## Group Operations

### split-apply-combine

In [2]:
# Example
rng = np.random.default_rng(seed=12345)
df = pd.DataFrame({"key1" : ["a","a",None,"b","b","a",None],
                   "key2" : pd.Series([1,2,1,2,1,None,1], dtype="Int64"),
                   "data1" : rng.standard_normal(7),
                   "data2" : rng.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


## Split -> groupby -> will create **intermediate grouped object**
## apply -> certain function
## combine -> get final output

In [3]:
# Access data1 and call groupby with the key1 column
# Create an intermediate grouped object -> wanted to do sth else
grouped = df.groupby("key1")["data1"]

In [4]:
# Compute mean
# If output has sth: "dtype: float64" -> Series
grouped.mean()

key1
a   -0.300327
b   -0.167258
Name: data1, dtype: float64

**By default**, pandas will:
1. **Sort group labels alphabetically**
2. **Exclude rows where the grouping key is missing (`NaN`)**
3. **Return the grouping key as the index**

## **Explicitly controlling GroupBy behavior**

To gain full control over the output, pandas provides three important parameters:

- **`sort=False`** → preserves the original order of appearance
    - By default, pandas **sorts group labels**, which may **reorder our results**

- **`dropna=False`** → includes missing values as a valid group
    - Missing values are frequently **informative**, especially in real-world datasets.

- **`as_index=False`** → keeps grouping keys as regular columns

In [5]:
# Group by 'key1' and compute the mean of 'data1'
# while preserving row order, keeping missing keys, and returning a DataFrame
# Without intermediate grouped object
df.groupby(
    "key1",
    sort=False,
    dropna=False,
    as_index=False
)["data1"].mean()

,key1,data1
0,a,-0.300327
1,NaN,-1.119227
2,b,-0.167258


In [6]:
# Passing multiple groupby arrays of keys to find mean
means = df.groupby([df["key1"], df["key2"]]).mean()
means

data1     data2
key1 key2                    
a    1    -1.423825  0.648893
     2     1.263728  0.361058
b    1    -0.075343  0.968497
     2    -0.259173  2.347410

In [7]:
# Passing multiple groupby arrays of keys to find mean
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -1.423825
      2       1.263728
b     1      -0.075343
      2      -0.259173
Name: data1, dtype: float64

In [8]:
# Unstack : from vertical to wide table
means.unstack()

key2,1,2
key1,,
a,-1.423825,1.263728
b,-0.075343,-0.259173


In [9]:
# Pass column names as the group keys
# i want to groupby all the data using key1 as the label
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,-0.300327,0.083521
b,1.5,-0.167258,1.657953


In [10]:
# Groupby using multiple keys -> by providing a list
df.groupby(["key1","key2"]).mean()

data1     data2
key1 key2                    
a    1    -1.423825  0.648893
     2     1.263728  0.361058
b    1    -0.075343  0.968497
     2    -0.259173  2.347410

In [11]:
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


In [12]:
# Returns a Series containing group sizes
df.groupby(["key1","key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [13]:
# Missing values in a group key are excluded from the result by default
# Can be disabled by passing dropna=False to groupby
# dropna=True is by default
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [14]:
# can use count as well
# but no argument from dropna. suggest to use size
df.groupby(["key1","key2"])["data1"].count()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
Name: data1, dtype: int64

In [15]:
# Returns a Series containing group sizes
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

### **`count()`**: Returns the number of non-null values in each group **for each column**.

In [16]:
# Difference between size() and count()
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})

print("DataFrame:")
print(df)
print("\nsize():")
print(df.groupby("key1").size())
print("\ncount():")
print(df.groupby("key1").count())

DataFrame:
   key1  key2     data1     data2
0     a     1  1.213533 -1.703515
1     a     2  1.414959 -1.581375
2  None     1  1.627631  0.376882
3     b     2 -0.772794 -1.122460
4     b     1 -0.514527  0.050847
5     a  <NA> -0.168475  1.102468
6  None     1  0.085323 -1.157249

size():
key1
a    3
b    2
dtype: int64

count():
      key2  data1  data2
key1                    
a        2      3      3
b        2      2      2


### **Iterating over Groups**
***groupby*** supports ***iteration***, generating ***a sequence of 2-tuples***

In [17]:
df

,key1,key2,data1,data2
0,a,1,1.213533,-1.703515
1,a,2,1.414959,-1.581375
2,None,1,1.627631,0.376882
3,b,2,-0.772794,-1.122460
4,b,1,-0.514527,0.050847
5,a,<NA>,-0.168475,1.102468
6,None,1,0.085323,-1.157249


## **`name` and `grp` is a placeholder**
- holding some information

In [18]:
# Iteration
# key 1 -> 'a' & 'b'
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  1.213533 -1.703515
1    a     2  1.414959 -1.581375
5    a  <NA> -0.168475  1.102468
b
  key1  key2     data1     data2
3    b     2 -0.772794 -1.122460
4    b     1 -0.514527  0.050847


In [19]:
# Computing a dictionary using one-liner
# for placeholder, I want to make them become dictionary
pieces = {name: group for name, group in df.groupby("key1")}

In [20]:
# Accessing the key "a" in dictionary
# get back the value
pieces["a"]

,key1,key2,data1,data2
0,a,1,1.213533,-1.703515
1,a,2,1.414959,-1.581375
5,a,<NA>,-0.168475,1.102468


In [21]:
# Accessing item in dictionary
pieces["b"]

,key1,key2,data1,data2
3,b,2,-0.772794,-1.122460
4,b,1,-0.514527,0.050847


### **Selecting a Column or Subset of Columns**

In [22]:
# Returned a DataFrame if a list or array is passed
# If use double square bracket, the output would be a pandas dataframe, dataframe can straightaway do visualization
df.groupby(["key1","key2"])[["data2"]].mean()

data2
key1 key2          
a    1    -1.703515
     2    -1.581375
b    1     0.050847
     2    -1.122460

In [23]:
# Returned a grouped Series if only column name is passed as a scalar
# scalar -> a single column name string
# single bracket is a series
df.groupby(["key1", "key2"])["data2"].mean()

key1  key2
a     1      -1.703515
      2      -1.581375
b     1       0.050847
      2      -1.122460
Name: data2, dtype: float64

### **Grouping with Dictionaries and Series**

In [24]:
# Example DataFrame
# Setting the seed number to ensure reproducibility
rng = np.random.default_rng(seed=12345)
people = pd.DataFrame(rng.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,0.361058,-1.952863
Wanda,2.347410,0.968497,-0.759387,0.902198,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,1.322298,-0.299699,0.902919,-1.621583,-0.158189


In [25]:
# Add a few NA values using loc function
# loc -> based on label
# iloc -> based on integer
people.loc["Wanda", ["b", "c"]] = np.nan
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,0.361058,-1.952863
Wanda,2.347410,NaN,NaN,0.902198,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,1.322298,-0.299699,0.902919,-1.621583,-0.158189


In [26]:
# Add a few NA values using iloc function
# single bracket -> indicate a list
# (4, 1) -> refer to rows
# (0, 3) -> refer to columns
people.iloc[(4, 1), (0, 3)] = np.nan
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,NaN,-1.952863
Wanda,2.347410,NaN,NaN,0.902198,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,NaN,-0.299699,0.902919,-1.621583,-0.158189


In [27]:
# Group correspondence for the columns
# Act as a dictionary
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange", "g" : "violet"}
mapping

{'a': 'red',
 'b': 'red',
 'c': 'blue',
 'd': 'blue',
 'e': 'red',
 'f': 'orange',
 'g': 'violet'}

In [28]:
# Using the mapping function
# `T` -> transpose
# debugging the code
people.T.groupby(mapping).sum().T

,blue,red
Joe,-1.129835,-0.235440
Steve,0.648893,-4.061540
Wanda,0.902198,1.880456
Jill,-0.680811,2.127134
Trey,-0.718663,-0.457888


In [29]:
# Add the column f and g
rng = np.random.default_rng(seed=12345)
people = people.assign(f = rng.standard_normal((5, 1)),
                       g = rng.standard_normal((5, 1)))
people

,a,b,c,d,e,f,g
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343,-1.423825,-0.740885
Steve,-0.740885,-1.367793,0.648893,NaN,-1.952863,1.263728,-1.367793
Wanda,2.347410,NaN,NaN,0.902198,-0.466953,-0.870662,0.648893
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979,-0.259173,0.361058
Trey,NaN,-0.299699,0.902919,-1.621583,-0.158189,-0.075343,-1.952863


In [30]:
# Run sum function again
people.T.groupby(mapping).sum().T

,blue,orange,red,violet
Joe,-1.129835,-1.423825,-0.235440,-0.740885
Steve,0.648893,1.263728,-4.061540,-1.367793
Wanda,0.902198,-0.870662,1.880456,0.648893
Jill,-0.680811,-0.259173,2.127134,0.361058
Trey,-0.718663,-0.075343,-0.457888,-1.952863


### **Grouping with Functions**

In [31]:
# Display people content
people

,a,b,c,d,e,f,g
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343,-1.423825,-0.740885
Steve,-0.740885,-1.367793,0.648893,NaN,-1.952863,1.263728,-1.367793
Wanda,2.347410,NaN,NaN,0.902198,-0.466953,-0.870662,0.648893
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979,-0.259173,0.361058
Trey,NaN,-0.299699,0.902919,-1.621583,-0.158189,-0.075343,-1.952863


In [32]:
# Groupby length of the index letter
people.groupby(len).sum()

,a,b,c,d,e,f,g
3,-1.423825,1.263728,-0.870662,-0.259173,-0.075343,-1.423825,-0.740885
4,-0.060690,0.489146,-0.353749,-1.045725,1.240790,-0.334517,-1.591805
5,1.606525,-1.367793,0.648893,0.902198,-2.419816,0.393067,-0.718900


# **13.2 Data Aggregation**
- get some summary data

In [33]:
# From previous example
df

,key1,key2,data1,data2
0,a,1,1.213533,-1.703515
1,a,2,1.414959,-1.581375
2,None,1,1.627631,0.376882
3,b,2,-0.772794,-1.122460
4,b,1,-0.514527,0.050847
5,a,<NA>,-0.168475,1.102468
6,None,1,0.085323,-1.157249


In [34]:
# Selects the smallest requested number
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     5   -0.168475
      0    1.213533
b     3   -0.772794
      4   -0.514527
Name: data1, dtype: float64

In [35]:
# Define own aggregation functions
def peak_2_peak(arr):
    return arr.max() - arr.min()

In [36]:
# agg: aggregate
# apply the function to the dataframe
grouped.agg(peak_2_peak)

,key2,data1,data2
key1,,,
a,1,1.583434,2.805983
b,1,0.258266,1.173306


### **`describe()`**: generates **descriptive statistics** for each group in the grouped object.
- provides **a summary of the data**

In [37]:
# Another way
# describe() -> quick and easy view of our dataset
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.820006  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.643661  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     1.314246  1.414959   3.0 -0.727474  1.585953 -1.703515 -1.642445   
b    -0.579094 -0.514527   2.0 -0.535807  0.829653 -1.122460 -0.829133   

                                    
           50%       75%       max  
key1                                
a    -1.581375 -0.239453  1.102468  
b    -0.535807 -0.242480  0.050847  

[2 rows x 24 columns]

## **Column-Wise and Multiple Function Application**

In [38]:
# Get the tipping dataset from github
# Download here: https://bit.ly/3VyE0vP
tips = pd.read_csv("https://bit.ly/3VyE0vP")
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [39]:
# add additional column
tips['tip_pct'] =  tips['tip']/tips['total_bill']
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [40]:
# create intermediate grouped object
# group the tips by day and smoker
grouped = tips.groupby(['day','smoker'])
grouped.describe()

total_bill                                                        \
                 count       mean        std    min     25%     50%      75%   
day  smoker                                                                    
Fri  No            4.0  18.420000   5.059282  12.46  15.100  19.235  22.5550   
     Yes          15.0  16.813333   9.086388   5.75  11.690  13.420  18.6650   
Sat  No           45.0  19.661778   8.939181   7.25  14.730  17.820  20.6500   
     Yes          42.0  21.276667  10.069138   3.07  13.405  20.390  26.7925   
Sun  No           57.0  20.506667   8.130189   8.77  14.780  18.430  25.0000   
     Yes          19.0  24.120000  10.442511   7.25  17.165  23.100  32.3750   
Thur No           45.0  17.113111   7.721728   7.51  11.690  15.950  20.2700   
     Yes          17.0  19.190588   8.355149  10.34  13.510  16.470  19.8100   

                     tip            ...  size      tip_pct            \
               max count      mean  ...   75%  max   count      mean   
day  smoker                         ...                                
Fri  No      22.75   4.0  2.812500  ...  2.25  3.0     4.0  0.151650   
     Yes     40.17  15.0  2.714000  ...  2.00  4.0    15.0  0.174783   
Sat  No      48.33  45.0  3.102889  ...  3.00  4.0    45.0  0.158048   
     Yes     50.81  42.0  2.875476  ...  3.00  5.0    42.0  0.147906   
Sun  No      48.17  57.0  3.167895  ...  4.00  6.0    57.0  0.160113   
     Yes     45.35  19.0  3.516842  ...  3.00  5.0    19.0  0.187250   
Thur No      41.19  45.0  2.673778  ...  2.00  6.0    45.0  0.160298   
     Yes     43.11  17.0  3.030000  ...  2.00  4.0    17.0  0.163863   

                                                                         
                  std       min       25%       50%       75%       max  
day  smoker                                                              
Fri  No      0.028123  0.120385  0.137239  0.149241  0.163652  0.187735  
     Yes     0.051293  0.103555  0.133739  0.173913  0.209240  0.263480  
Sat  No      0.039767  0.056797  0.136240  0.150152  0.183915  0.291990  
     Yes     0.061375  0.035638  0.091797  0.153624  0.190502  0.325733  
Sun  No      0.042347  0.059447  0.139780  0.161665  0.185185  0.252672  
     Yes     0.154134  0.065660  0.097723  0.138122  0.215325  0.710345  
Thur No      0.038774  0.072961  0.137741  0.153492  0.184843  0.266312  
     Yes     0.039389  0.090014  0.148038  0.153846  0.194837  0.241255  

[8 rows x 32 columns]

In [41]:
# Get descriptive statistics
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

## **Difficult to understand the table output without further investigation**

In [42]:
# Getting back a DataFrame
# Multiple function application
grouped_pct.agg(["mean", "std", peak_2_peak])

mean       std  peak_2_peak
day  smoker                                 
Fri  No      0.151650  0.028123     0.067349
     Yes     0.174783  0.051293     0.159925
Sat  No      0.158048  0.039767     0.235193
     Yes     0.147906  0.061375     0.290095
Sun  No      0.160113  0.042347     0.193226
     Yes     0.187250  0.154134     0.644685
Thur No      0.160298  0.038774     0.193350
     Yes     0.163863  0.039389     0.151240

## **Named aggregation helped to solve this problem**
- by giving **clear column names**

In [43]:
# Named aggregation (clearer output column names)
grouped_pct.agg(
    mean_tip_pct="mean",
    std_tip_pct="std",
    range=peak_2_peak
)

mean_tip_pct  std_tip_pct     range
day  smoker                                     
Fri  No          0.151650     0.028123  0.067349
     Yes         0.174783     0.051293  0.159925
Sat  No          0.158048     0.039767  0.235193
     Yes         0.147906     0.061375  0.290095
Sun  No          0.160113     0.042347  0.193226
     Yes         0.187250     0.154134  0.644685
Thur No          0.160298     0.038774  0.193350
     Yes         0.163863     0.039389  0.151240

In [44]:
# Specify a list of functions to apply
# by providing a list
funcs = ["count", "mean", "max", "min", "median"]
result = grouped[["tip_pct", "total_bill"]].agg(funcs)
result

tip_pct                                         total_bill  \
              count      mean       max       min    median      count   
day  smoker                                                              
Fri  No           4  0.151650  0.187735  0.120385  0.149241          4   
     Yes         15  0.174783  0.263480  0.103555  0.173913         15   
Sat  No          45  0.158048  0.291990  0.056797  0.150152         45   
     Yes         42  0.147906  0.325733  0.035638  0.153624         42   
Sun  No          57  0.160113  0.252672  0.059447  0.161665         57   
     Yes         19  0.187250  0.710345  0.065660  0.138122         19   
Thur No          45  0.160298  0.266312  0.072961  0.153492         45   
     Yes         17  0.163863  0.241255  0.090014  0.153846         17   

                                              
                  mean    max    min  median  
day  smoker                                   
Fri  No      18.420000  22.75  12.46  19.235  
     Yes     16.813333  40.17   5.75  13.420  
Sat  No      19.661778  48.33   7.25  17.820  
     Yes     21.276667  50.81   3.07  20.390  
Sun  No      20.506667  48.17   8.77  18.430  
     Yes     24.120000  45.35   7.25  23.100  
Thur No      17.113111  41.19   7.51  15.950  
     Yes     19.190588  43.11  10.34  16.470

In [45]:
# Extract certain column
result["total_bill"]

count       mean    max    min  median
day  smoker                                        
Fri  No          4  18.420000  22.75  12.46  19.235
     Yes        15  16.813333  40.17   5.75  13.420
Sat  No         45  19.661778  48.33   7.25  17.820
     Yes        42  21.276667  50.81   3.07  20.390
Sun  No         57  20.506667  48.17   8.77  18.430
     Yes        19  24.120000  45.35   7.25  23.100
Thur No         45  17.113111  41.19   7.51  15.950
     Yes        17  19.190588  43.11  10.34  16.470

## **Using tuple**
- non-immutable
- **it has an sequential order**

In [46]:
# Passing a list of tuples
ftuples = [("Average", "mean"), ("Variation", "var")]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average Variation    Average   Variation
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [47]:
# Original code
# very lengthy and hard to "see"
grouped.agg(tip_max=('tip', 'max'),size_sum=('size', 'sum'))

tip_max  size_sum
day  smoker                   
Fri  No         3.50         9
     Yes        4.73        31
Sat  No         9.00       115
     Yes       10.00       104
Sun  No         6.00       167
     Yes        6.50        49
Thur No         6.70       112
     Yes        5.00        40

In [48]:
# Implicit line continuation
# Wrap the code in parenthesis -> "()"
# Any code wrapped in parenthesis, can be broken into different rows
(
    grouped
    .agg(tip_max=('tip', 'max'),
         size_sum=('size', 'sum'))
)

tip_max  size_sum
day  smoker                   
Fri  No         3.50         9
     Yes        4.73        31
Sat  No         9.00       115
     Yes       10.00       104
Sun  No         6.00       167
     Yes        6.50        49
Thur No         6.70       112
     Yes        5.00        40

In [49]:
# Another fancy way
grouped.agg({"total_bill" : ["min", "max", "mean", "std"],
             "size" : "sum"})

total_bill                              size
                   min    max       mean        std  sum
day  smoker                                             
Fri  No          12.46  22.75  18.420000   5.059282    9
     Yes          5.75  40.17  16.813333   9.086388   31
Sat  No           7.25  48.33  19.661778   8.939181  115
     Yes          3.07  50.81  21.276667  10.069138  104
Sun  No           8.77  48.17  20.506667   8.130189  167
     Yes          7.25  45.35  24.120000  10.442511   49
Thur No           7.51  41.19  17.113111   7.721728  112
     Yes         10.34  43.11  19.190588   8.355149   40

In [50]:
# Implicit line continuation
(
    grouped[["tip_pct", "total_bill"]]
    .agg(["count", "mean", "median"])
    .rename_axis(index=["day", "smoker"])
)


tip_pct                     total_bill                   
              count      mean    median      count       mean  median
day  smoker                                                          
Fri  No           4  0.151650  0.149241          4  18.420000  19.235
     Yes         15  0.174783  0.173913         15  16.813333  13.420
Sat  No          45  0.158048  0.150152         45  19.661778  17.820
     Yes         42  0.147906  0.153624         42  21.276667  20.390
Sun  No          57  0.160113  0.161665         57  20.506667  18.430
     Yes         19  0.187250  0.138122         19  24.120000  23.100
Thur No          45  0.160298  0.153492         45  17.113111  15.950
     Yes         17  0.163863  0.153846         17  19.190588  16.470

## **Returning Aggregated Data Without Row Indexes**
- Using **`as_index`=False/True**

In [51]:
# Disable index
(tips
 .groupby(["day", "smoker"], as_index=False)
 [['total_bill', 'tip', 'size', 'tip_pct']]
 .mean()
)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# **13.3 Apply: General split-apply-combine**

In [52]:
# Define our own "top" function call
# Define a function to select the top five tip_pct values by group
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

# Overwrite the default value
top(tips, n=3)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733


## Using our own function has **one caveat**
- in terms of **speed**, may be it is **not that "efficient"**
- compared to **native python code**

In [53]:
# Direct chaining
# instead of using the function "top"
# we use head() -> native python function
tips.sort_values("tip_pct", ascending=False).head(3)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733


## **Benchmarking code**
We will **almost certainly** observe (**in terms of execution speed**):

`Direct chaining`  <  `Function version`

In [55]:
import timeit

#number of repetitions
n_runs=1000

# Timing Code A (function call)
time_func = timeit.timeit(
    stmt = "top(tips,n=6)",
    globals=globals(),
    number=n_runs
)

# Timing Code B (function call)
time_func = timeit.timeit(
    stmt = 'tips.sort_values("tip_pct", ascending=False).head(6)',
    globals=globals(),
    number=n_runs
)

print(f"Function version: {time_func:.6f} seconds")
print(f"Direct chaining: {time_direct:.6f} seconds")

Function version: 0.430135 seconds


NameError: name 'time_direct' is not defined

In [56]:
# display tips dataframe
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [58]:
# Apply function on certain selected columns
tips.groupby("smoker")[['total_bill','tip','size','tip_pct']].apply(top)

total_bill   tip  size   tip_pct
smoker                                      
No     232       11.61  3.39     2  0.291990
       149        7.51  2.00     2  0.266312
       51        10.29  2.60     2  0.252672
       185       20.69  5.00     5  0.241663
       88        24.71  5.85     2  0.236746
Yes    172        7.25  5.15     2  0.710345
       178        9.60  4.00     2  0.416667
       67         3.07  1.00     1  0.325733
       183       23.17  6.50     4  0.280535
       109       14.31  4.00     2  0.279525

In [59]:
# Overwrite the default values
(
    tips
    .groupby("smoker")[['total_bill', 'tip', 'size', 'tip_pct']]
    .apply(top, n=3, column="total_bill")
)

total_bill    tip  size   tip_pct
smoker                                       
No     212       48.33   9.00     4  0.186220
       59        48.27   6.73     4  0.139424
       156       48.17   5.00     6  0.103799
Yes    170       50.81  10.00     3  0.196812
       182       45.35   3.50     3  0.077178
       102       44.30   2.50     3  0.056433

In [60]:
# Earlier example
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [61]:
# Unstack() -> from wide to vertical
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

## **Filling Missing Values with Group-Specific Values**

In [62]:
# Create a mock data
rng = np.random.default_rng(seed=12345)
s =pd.Series(rng.standard_normal(6))
s

0   -1.423825
1    1.263728
2   -0.870662
3   -0.259173
4   -0.075343
5   -0.740885
dtype: float64

In [63]:
# Skip of 2
s.iloc[::2] = np.nan
s

0         NaN
1    1.263728
2         NaN
3   -0.259173
4         NaN
5   -0.740885
dtype: float64

In [66]:
# Fill NA with mean
s.fillna(s.mean())

0    0.087890
1    1.263728
2    0.087890
3   -0.259173
4    0.087890
5   -0.740885
dtype: float64

In [67]:
# Another example data
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(rng.standard_normal(8), index=states)
data

Ohio         -1.367793
New York      0.648893
Vermont       0.361058
Florida      -1.952863
Oregon        2.347410
Nevada        0.968497
California   -0.759387
Idaho         0.902198
dtype: float64

In [68]:
# Insert NaN value into the series
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio         -1.367793
New York      0.648893
Vermont            NaN
Florida      -1.952863
Oregon        2.347410
Nevada             NaN
California   -0.759387
Idaho              NaN
dtype: float64

In [69]:
# Extract size
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [70]:
# Using count function
# count() function deals only with non-null value
# will ignore Nan value
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [71]:
# Original shape
data

Ohio         -1.367793
New York      0.648893
Vermont            NaN
Florida      -1.952863
Oregon        2.347410
Nevada             NaN
California   -0.759387
Idaho              NaN
dtype: float64

## This is using **`transform`** function
- `transform` function **retained the original shape** of dataset

In [72]:
# Fill NA using group mean
# preferred: transform is vectorized and aligns to original rows
data = data.fillna(
    data.groupby(group_key).transform("mean")
)
data

Ohio         -1.367793
New York      0.648893
Vermont      -0.890588
Florida      -1.952863
Oregon        2.347410
Nevada        0.794011
California   -0.759387
Idaho         0.794011
dtype: float64

## This is using **`apply`** function
- will **return as much information as possible**

In [73]:
# Fill the NA values using the group means
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio         -1.367793
      New York      0.648893
      Vermont      -0.890588
      Florida      -1.952863
West  Oregon        2.347410
      Nevada        0.794011
      California   -0.759387
      Idaho         0.794011
dtype: float64

## **Caveat here: `transform` vs `apply`**
- in terms of **execution speed**: **`transform`** < **`apply`**

In [75]:
# Another example data
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(rng.standard_normal(8), index=states)
data

Ohio         -0.466953
New York     -0.060690
Vermont       0.788844
Florida      -1.256668
Oregon        0.575858
Nevada        1.398979
California    1.322298
Idaho        -0.299699
dtype: float64

In [76]:
# Insert NaN value into the series
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio         -0.466953
New York     -0.060690
Vermont            NaN
Florida      -1.256668
Oregon        0.575858
Nevada             NaN
California    1.322298
Idaho              NaN
dtype: float64

In [77]:
# With predefined fill values
fill_values = {"East": 0.5, "West": -1}

def fill_func(group):
    return group.fillna(fill_values[group.name])

In [78]:
# Filling in the NA with predefined value
data.groupby(group_key).apply(fill_func)

East  Ohio         -0.466953
      New York     -0.060690
      Vermont       0.500000
      Florida      -1.256668
West  Oregon        0.575858
      Nevada       -1.000000
      California    1.322298
      Idaho        -1.000000
dtype: float64

# **13.4 Group Transforms and “Unwrapped” GroupBys**
- looking at **in terms of execution speed**

In [79]:
# An example data
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [80]:
# Group means by key
# Create an intermediate grouped object
g = df.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [81]:
# Define a mean function
def get_mean(g):
    return g.mean()

In [82]:
# Transform into a Series of mean data
# try to get back the original shape
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [83]:
# Another way - this is faster using default function
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [84]:
# A normalization function
def normalize(x):
    return (x - x.mean()) / x.std()

In [85]:
# Method 1 - faster
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [86]:
# Method 2: apply - slower
# index of the resulting Series will be a MultiIndex
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

## **Unwrapped function**
- straight away **without intermediate grouped object**
- almost certainly **the fastest** compared to `transform` and `apply`

In [88]:
# Unwrapped group operations - doing arithmetic between the outputs of
# multiple GroupBy operations
# often much faster than a general apply function
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

# **13.5 Pivot Tables and Cross-Tabulation**
A ***pivot table*** is a ***data summarization*** tool
- like Microsoft Excel

In [90]:
# From previous tipping dataset
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


## **Pivot table: three parameters**
1. `index`,
2. `columns`,
3. `values`

In [91]:
# Getting mean for size and tip_pct columns
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [92]:
# Use aggfunc = len for count
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [93]:
# pass NA with a specific value using fill_value
# placeholder / sentinel value
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=-999)

day                        Fri         Sat         Sun        Thur
time   size smoker                                                
Dinner 1    No     -999.000000    0.137931 -999.000000 -999.000000
            Yes    -999.000000    0.325733 -999.000000 -999.000000
       2    No        0.139622    0.162705    0.168859    0.159744
            Yes       0.171297    0.148668    0.207893 -999.000000
       3    No     -999.000000    0.154661    0.152663 -999.000000
            Yes    -999.000000    0.144995    0.152660 -999.000000
       4    No     -999.000000    0.150096    0.148143 -999.000000
            Yes       0.117750    0.124515    0.193370 -999.000000
       5    No     -999.000000 -999.000000    0.206928 -999.000000
            Yes    -999.000000    0.106572    0.065660 -999.000000
       6    No     -999.000000 -999.000000    0.103799 -999.000000
Lunch  1    No     -999.000000 -999.000000 -999.000000    0.181728
            Yes       0.223776 -999.000000 -999.000000 -999.000000
       2    No     -999.000000 -999.000000 -999.000000    0.166005
            Yes       0.181969 -999.000000 -999.000000    0.158843
       3    No        0.187735 -999.000000 -999.000000    0.084246
            Yes    -999.000000 -999.000000 -999.000000    0.204952
       4    No     -999.000000 -999.000000 -999.000000    0.138919
            Yes    -999.000000 -999.000000 -999.000000    0.155410
       5    No     -999.000000 -999.000000 -999.000000    0.121389
       6    No     -999.000000 -999.000000 -999.000000    0.173706

## **Cross-Tabulations: Crosstab**
A ***special pivot table*** to compute ***group frequencies***

In [94]:
from io import StringIO

In [95]:
# Data consists of strings
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

### **`sep=r"\s+"`**
- "one or more whitespace characters"

### also `sep=\t, csv`

In [97]:
# Create a table
data = pd.read_table(StringIO(data), sep=r"\s+")

In [98]:
# Display the data content
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [99]:
# Crosstab Example 1 - find group frequencies
# Summarize by nationality and handedness
# Crosstab: row and column
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [100]:
# Tipping dataset
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [103]:
# Crosstab Example 2
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

# **Closing Recap: Choosing the Right Tool in Grouped Analysis**

### When to use `agg` vs `transform` vs `apply`

**Use `agg()`** when you want **one row per group** (summary tables, reporting).
- Fast and optimized.
- Best for group-level statistics.

**Use `transform()`** when you want **the same number of rows as the original data** (group-wise calculations).
- Ideal for normalization, filling NA by group, ranking within group.
- Vectorized and typically faster than `apply()`.

**Use `apply()`** only when the logic **cannot be expressed** using `agg()` or `transform()`.
- Most flexible, but usually slower.
- Prefer avoiding it for standard tasks like top-N or filling missing values.

---

# **When to use `pivot_table()` vs `groupby().agg().unstack()`**

**Use `pivot_table()`** for quick, Excel-style summaries and teaching-friendly cross-tab summaries.
- Concise syntax.
- Handles missing combinations cleanly.

**Use `groupby().agg().unstack()`** when you want more explicit, step-by-step control.
- Better for complex pipelines and debugging.
- More transparent for production code.
- wide to vertical table, and vice versa
